In [4]:
import numpy as np
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem,get_termination
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.model.problem import Problem
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

In [5]:
#defining problem
class AdaptationProblem(Problem):
    """
    response times are filled based on papers values(page8-table1)
    """
    def __init__(self,
                landa=50, #arrival rate (req/s)
                n=10, #average data payload for each request (KB)
                p_i=2.64E-04, #cost of a VM ($/s)
                p_n=3.50E-06, #cost of data transfer ($/KB)
                H=8.50E-05, #static Hosting cost ($/s)
                RPM=0.7, #revenue per 1000 ads ($)
                R=100, #response time (ms)
                gamma_l=1, #gamma lower bound (gamma is average number of ad banners per page)
                gamma_u=10, #gamma upper bound
                R_l=45, # respose time lower bound
                R_u=200, # response time upper bound
                d_l=15, #capacity of each VM lower bound(request/s)
                d_u=21 #capacity of each VM upper bound(request/s)
                ): 
        self.landa = landa
        self.n = n
        self.p_i = p_i
        self.p_n = p_n
        self.H = H
        self.RPM = RPM
        self.R = R
        self.gamma_l = gamma_l
        self.gamma_u = gamma_u
        self.R_l = R_l
        self.R_u = R_u
        self.d_l = d_l
        self.d_u = d_u
        #weights in user satisfaction formula
        self.a = 0.5
        self.b = 0.5
        #p_s doesn't have any bound so we should feed algorithm 
        #a very high number for upper bound and 0 (logically) for lower bound
        super().__init__(n_var=3, #p_s,W,gamma
                         n_obj=3, #pi_s,pi_a,U
                         n_constr=4,
                         xl=np.array([0,landa/d_u,gamma_l]),
                         xu=np.array([10000,landa/d_l,gamma_u]))

    def _evaluate(self, x, out, *args, **kwargs):
        #objectives
        #to maximize objectives we multipy them by -1 and then minimize them 
        #objectives are not normalized. test normalization on results
        d = self.landa/x[:,1]
        rel_d = (d - self.d_l) / (self.d_u - self.d_l)
        R = ((self.R_u-self.R_l)*rel_d) + self.R_l
        pi_s = x[:,0]*self.landa - self.p_i*x[:,1] - self.n*self.p_n*self.landa
        pi_a = (x[:,2]*self.RPM/1000)*self.landa - self.H*self.landa - x[:,0]*self.landa
        U = self.a*((self.gamma_u-x[:,2])/(self.gamma_u-self.gamma_l)) + self.b*((self.R_u-R)/(self.R_u-self.R_l))
        
        out["F"] = np.column_stack([-1*pi_s, -1*pi_a,-1*U])
            
        #constrains (equations are filped in a way that they would be less than zero)
        g1 = x[:,0] - (x[:,2]*self.RPM/1000)
        g2 = -1*(pi_s)
        g3 = -1*(pi_a)
        g4 = (0.8 - U)

        out["G"] = np.column_stack([g1, g2, g3, g4])


In [6]:
#defining algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    eliminate_duplicates=True
)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.configuration import Configuration
Configuration.show_compile_hint = False



In [12]:
#defining termination
termination = get_termination('n_gen',150)

In [13]:
#running minimize problem
problem = AdaptationProblem(R=1918,n=589,landa=52)

res = minimize(problem,
               algorithm,
               termination,
               save_history=True,
               verbose=True
              )

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  | delta_ideal  | delta_nadir  |   delta_f   
    1 |     100 |  1.76442E+04 |  2.56189E+05 |       1 |            - |            - |            -
    2 |     110 |  1.62590E+04 |  2.15364E+05 |       1 |  1.35910E+03 |  1.35910E+03 |  1.92191E+33
    3 |     120 |  1.62590E+04 |  1.84318E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    4 |     130 |  1.62590E+04 |  1.55000E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    5 |     140 |  1.62590E+04 |  1.30586E+05 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    6 |     150 |  1.35928E+04 |  1.11122E+05 |       1 |  2.61615E+03 |  2.61615E+03 |  3.69964E+33
    7 |     160 |  7.18853E+03 |  9.25539E+04 |       1 |  6.28338E+03 |  6.28338E+03 |  8.88600E+33
    8 |     170 |  7.18853E+03 |  7.95997E+04 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
    9 |     180 |  7.18853E+03 |  6.71805E+04 |       1 |  0.00000E+00 |  0.00000E+00 |  0.

   83 |     920 |  0.141108282 |  0.828880846 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   84 |     930 |  0.141108282 |  0.722271753 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   85 |     940 |  0.141108282 |  0.659966991 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   86 |     950 |  0.141108282 |  0.608096859 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   87 |     960 |  0.141108282 |  0.573729089 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   88 |     970 |  0.141108282 |  0.563810566 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   89 |     980 |  0.141108282 |  0.539207643 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   90 |     990 |  0.141108282 |  0.495144169 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   91 |    1000 |  0.141108282 |  0.462598855 |       1 |  0.00000E+00 |  0.00000E+00 |  0.00000E+00
   92 |    1010 |  0.141108282 |  0.437882438 |       1 |  0.00000E+00 |  0.00000E+00 |  0.

In [14]:
res.X

array([[2.18392444e-03, 3.32797238e+00, 3.36736187e+00],
       [2.19637455e-03, 3.29043374e+00, 3.28984956e+00],
       [2.18741185e-03, 3.29043374e+00, 3.26599707e+00],
       [2.18101133e-03, 3.46175821e+00, 3.29892388e+00],
       [2.18100991e-03, 3.32797238e+00, 3.36741764e+00],
       [2.10970935e-03, 3.31156779e+00, 3.33363856e+00],
       [2.18392444e-03, 3.32827908e+00, 3.36736187e+00],
       [2.18101133e-03, 3.46453493e+00, 3.29892388e+00],
       [2.16989580e-03, 3.44580286e+00, 3.34944013e+00]])

In [ ]:



fig = pyplot.figure()
ax = Axes3D(fig)
f = -1*res.F

sequence_containing_x_vals = f[:,0]
sequence_containing_y_vals = f[:,1]
sequence_containing_z_vals = f[:,2]

random.shuffle(sequence_containing_x_vals)
random.shuffle(sequence_containing_y_vals)
random.shuffle(sequence_containing_z_vals)

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

In [ ]:
# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]


In [ ]:
X = np.array([pop.get("X") for pop in pop_each_gen])
G = np.array([pop.get("G") for pop in pop_each_gen])
F = np.array([pop.get("F") for pop in pop_each_gen])

In [ ]:
gen = 129

In [ ]:
plt.scatter(G[gen][:,0],G[gen][:,1])
plt.xlabel("p_s - y*RPM/1000")
plt.ylabel("-pi_s")

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(G[gen][:,2],G[gen][:,3])
plt.xlabel("-pi_a")
plt.ylabel("0.8-U")

In [ ]:
plt.scatter(X[gen][:,0],X[gen][:,1])
plt.xlabel("p_s")
plt.ylabel("W")

In [ ]:
plt.scatter(X[gen][:,1],X[gen][:,2])
plt.xlabel("W")
plt.ylabel("gamma")

In [ ]:
pr = AdaptationProblem(landa=250,n=0.1,R_l=1,R_u=200,d_l=10,d_u=200)

In [ ]:
res = minimize(pr,
               algorithm,
               termination,
               save_history=True,
               verbose=True
              )

In [ ]:

def choose_on_pf(pareto_f,design_space,w_s=0.33,w_a=0.33,w_u=0.33):
    max_values = np.max(pareto_f,axis=0)
    arg_max = np.argmax(pareto_f[:,0]*w_s/max_values[0] + pareto_f[:,1]*w_a/max_values[1] + pareto_f[:,2]*w_u/max_values[2])
    return tuple(design_space[arg_max])

In [ ]:
p_s , W , gamma = choose_on_pf(-1*res.F,res.X)